In [9]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

from IPython.display import clear_output
!ollama pull llama3.1:8b

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 CLI
############################################################################################# 100.0%
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕▏ 4.7 GB                         
pulling 11ce4ee3e170... 100% ▕▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                 

In [2]:
! pip install pymilvus openai

In [ ]:
!pip install grpcio==1.37.1

In [6]:
from pymilvus import Collection, connections
from openai import OpenAI

# Initialize OpenAI API client
client = OpenAI(api_key='')

def generate_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

def connect_milvus():
    connections.connect(host="abc.ca-central-1.compute.amazonaws.com", port="19530")

def query_milvus(text):
    query_embedding = generate_embedding(text)
    collection = Collection(name="textbook")
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param={"metric_type": "L2"},
        limit=1,
        output_fields=["id", "text"]
    )
    return results

def format_context(results):
    context = ""
    collection = Collection(name="textbook")
    for result in results:
        for hit in result:
            expr = f"id == {hit.id}"
            entities = collection.query(expr, output_fields=["text"])  # Only retrieve the 'text' field
            for entity in entities:
                context += entity.get("text", "") + "\n"
    return context.strip()  # Remove trailing newline if necessary


In [24]:
import time
# Function to generate a response using the pulled Llama model
def generate_response_ollama(prompt):
    # The URL of the running Ollama server
    ollama_url = "http://127.0.0.1:11434/v1/completions"

    # Prepare the data for the API request
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3.1:8b",
        "prompt": prompt,
        "max_tokens": 500,
    }

    # Send the request to the Ollama API
    response = requests.post(ollama_url, headers=headers, data=json.dumps(data))

    # Handle the response
    if response.status_code == 200:
        return response.json()["choices"][0]["text"].strip()
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage of the Ollama API to generate a response
connect_milvus()


def generate_answer(question):
    results = query_milvus(question)
    context = format_context(results)
    system_prompt = "You are a knowledgeable and patient AI tutor. Provide clear, detailed, and helpful explanations to the questions asked based on the context provided. provide formulae where necessary"
    prompt = f"Context: {context} Question: {question}\nAnswer:"

    # Generate the response using Ollama's API
    answer = generate_response_ollama(prompt)
    return answer



In [20]:
question = "Define stereo vision"
answer= generate_answer(question)
print("Question:", question)
print("Generated Answer:", answer)

Question: Define stereo vision
Generated Answer: Stereo vision refers to how humans perceive solid shapes through the disparity in images captured from different viewpoints. It is a way of seeing 3D structures from 2D images, where the difference in perspective between two or more views allows us to estimate depth information. In essence, it involves taking advantage of the fact that when we look at an object side by side with its reflection or with another view of itself, the difference in appearance (disparity) of features like edges and corners across the images tells us how far away that point is from us.


In [21]:
question = "What is the principle of stereo vision for 3D shape measurement?"
answer= generate_answer(question)
print("Question:", question)
print("Generated Answer:", answer)

Question: What is the principle of stereo vision for 3D shape measurement?
Generated Answer: The principle of stereo vision for 3D shape measurement is based on the idea that when an object is viewed from two different angles, the difference in the position of the object between the two views (known as disparity) is inversely proportional to the distance of the object from the observer. In other words, objects that are farther away appear smaller and have less disparity than objects that are closer to the observer.

Specifically, under simple imaging configurations (such as both eyes or cameras looking straight ahead), the disparity between two views is directly related to the depth (distance) of the object in the scene. This fundamental relationship has been utilized in computer vision for various applications such as 3D shape measurement, model building, and image-based rendering.

In essence, stereo vision relies on the principles of epipolar geometry, which describes how correspond

In [25]:
question = "What are intrinsic and extrinsic camera parameters and how are they obtained through calibration?"
answer= generate_answer(question)
print("Question:", question)
print("Generated Answer:", answer)

Question: What are intrinsic and extrinsic camera parameters and how are they obtained through calibration?
Generated Answer: Based on the context, here's an answer to your question:

**Intrinsic and Extrinsic Camera Parameters**

* **Intrinsic parameters**: These refer to the internal settings of a camera that affect the image it captures. They include:
	+ Focal length (f): distance between the image sensor and the lens
	+ Principal point (cc1, cc2): location of the optical center within the image sensor
	+ Distortion coefficients: parameters that describe radial or tangential distortion in the lenses
* **Extrinsic parameters**: These refer to the external position and orientation of a camera. They include:
	+ Translation (tx, ty, tz): movement away from the origin along each axis
	+ Rotation matrix (Rx, Ry, Rz): describes the 3D rotation around each axis

These parameters can be estimated through various calibration methods, such as:

1. **Camera calibration boards**: Using a known l

In [26]:
question = "What is the epipolar constraint and how does help in stereo image matching?"
answer= generate_answer(question)
print("Question:", question)
print("Generated Answer:", answer)

Question: What is the epipolar constraint and how does help in stereo image matching?
Generated Answer: Based on the context provided:

The epipolar constraint is that the corresponding point in the unrectified image must lie within the epipolar line, which runs through the image plane parallel to or skew with the projection of the viewing ray at infinity and bounded by the two corresponding epipoles. This constraint helps in stereo image matching by limiting the search for potential correspondences to a specific region in the other image, thereby reducing the number of potential correspondences and increasing the reliability of the matching process.

In other words, knowing the epipolar geometry allows us to filter out possibilities early on that an object or point can't possibly correspond to because it lies outside the epipolar line. With this constraint in place, we only need to search for corresponding points along the epipolar line segment corresponding to a given pixel, reducing